<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Install packages

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts", "torchtext", "datasets", "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

In [ ]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [1]:
!git -C dl_hub pull


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 342 bytes | 342.00 KiB/s, done.
From https://github.com/jyanivaddi/dl_hub
   d8a625d..134449a  main       -> origin/main
Updating d8a625d..134449a
Fast-forward
 Transformer/PL_model.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [2]:
import torch
import torch.nn as nn
#from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.config import get_config
from dl_hub.Transformer.PL_data_module import OpusDataModule
from dl_hub.Transformer.PL_model import LitTransformer
from dl_hub.Transformer.PL_main import train_transformer


In [3]:
cfg = get_config()
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 10
cfg["ds_mode"] = 'disk'
cfg["ds_path"] = "/home/e183534/OpusBooks"


**Define data module**

In [4]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()

**Define PL Model**

In [5]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt)
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)


/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss_criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_criterion'])`.
  rank_zero_warn(


In [6]:
train_transformer(transformer_model, opus_dm, cfg, epochs=cfg['num_epochs'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 75.1 M
----------------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/e183534/ERA_V1/venv

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: I sought my ideal of a woman amongst English ladies, French countesses, Italian signoras, and German grafinnen. I could not find her.
    TARGET: Cercai il mio ideale di donna fra le donne inglesi, fra le contesse francesi, fra le patrizie italiane, fra le nobili tedesche, senza trovarlo.
 PREDICTED: Non a me , e non , ma non , ma non mi .
*****************************************

*****************************************
    SOURCE: An old woman we met in the shop then kindly took us along with her for a quarter of a mile, to a lady friend of hers, who occasionally let rooms to gentlemen.
    TARGET: Una vecchia incontrata nella bottega ci condusse gentilmente con lei per un quarto di miglio da una donnetta amica sua, che talvolta affittava delle stanze a dei signori.
 PREDICTED: La sua cosa era un ’ altra , e la sua cosa , e la sua cosa , e la sua cosa , e la sua cosa , e la sua sua sua , e la sua , e la sua cosa .
**************

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: He saw nature--he saw books through me; and never did I weary of gazing for his behalf, and of putting into words the effect of field, tree, town, river, cloud, sunbeam--of the landscape before us; of the weather round us--and impressing by sound on his ear what light could no longer stamp on his eye.
    TARGET: Per mezzo mio vedeva e leggeva. Non mi stancavo mai di dipingergli i campi, i fiumi, le città, gli alberi, le nubi, i raggi del sole sui paesaggi e di dargli, con la parola quelle impressioni che gli occhi gli negavano.
 PREDICTED: Egli vedeva : la natura e non mi vide mai più , e mi misi a guardare l ' effetto del fiume , mentre si nella città , un ' atmosfera che avevamo nascosto dal cielo , e un suono simile al buio .
*****************************************

*****************************************
    SOURCE: Sometimes he sighed deeply and said, 'Oh, my God!'
    TARGET: A volte, mentre respirava con difficoltà, dice

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Fancies, memories, and most strange thoughts followed one another with extreme rapidity and clearness: now he saw himself pouring out medicine for the patient and overfilling the spoon, then he saw the midwife's white hands, or Karenin's curious pose as he knelt on the floor by her bedside.
    TARGET: Le immagini, le memorie e le idee più strane si susseguivano le une alle altre con straordinaria velocità e chiarezza: ora la medicina che aveva versato all’ammalata e che aveva fatto gocciolare dal cucchiaino, ora le braccia bianche della levatrice, o la strana posizione di Aleksej Aleksandrovic sul pavimento, davanti al letto.
 PREDICTED: , e più strano , più strano che l ’ unico pensiero si sentiva con la mente che egli si avvicinava alla medicina e poi , dopo aver preso il cucchiaio , si vide sulla tavola , e vide la voce di Aleksej Aleksandrovic che parlava come al letto .
*****************************************

**************

In [8]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.29' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by /home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (requ